In [2]:
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.read_csv('../../data/merged_data_sentiment.csv')
df.head()

,ID,Content,User,Date,Location,Reactions,N_Children,Post Title,Platfrom,meta,lang,Subreddit,Unnamed: 0,Sentiment
0,UgyDawiDJRU5j8e9g6x4AaABAg,i love my mum~!TAT,carol lui,2013-04-30,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCgxe0Tv...",en,NaN,NaN,"0.9998131394386292,0.9847044944763184,0.992429..."
1,Ugx5UKqZcGpX0Qi8art4AaABAg,I was in tears until the lady gave her mother ...,C Ladner,2013-05-13,NaN,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UC98Fh4X...",en,NaN,NaN,"-0.8607058525085449,-0.4791601002216339,0,0.0,..."
2,UgybIB4C2ppmGqU40WV4AaABAg,"I like her voice!! So deep, i thought the voic...",joseph jr.,2013-05-24,PH,0.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCUe_b0X...",en,NaN,NaN,"0.9991986155509949,0.9696046113967896,0.984618..."
3,Ugyi1lQIQnmZ2jVwk4B4AaABAg,A verve and flow equal to Maria Carrey's debut...,medusanerve,2013-05-28,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCILrk7Q...",en,NaN,NaN,"0.9990953207015991,0.901877224445343,0.9919101..."
4,Ugw2iIVWPilqFB7RMEl4AaABAg,great voice:D\n,ramon goedendorp,2013-06-08,NaN,1.0,0.0,AI - ママへ (short ver.) Music Video,youtube,"{'vid_id': 'JjVqw-rKqJo', 'user_id': 'UCNs25ls...",en,NaN,NaN,"0.9998722076416016,0.9386110305786133,0.989294..."


In [3]:
# only looking at rows with location
df = df[df['Location'].notna()]
df.shape

(6444, 14)

In [8]:
# explore location formats
df['Location'].value_counts().tail(20)

Alexandria, Egypt      1
Newcastle, England     1
Andromeda, Galaxy      1
Copenhagen, DK         1
Birmingham, AL         1
London, UK             1
Miami, FL              1
Agra, India            1
VA                     1
Waterford, Ireland     1
kaduna, ng             1
Bristol, Angleterre    1
Patna, India           1
DO                     1
Yelm, Washington       1
Honolulu, HI           1
Freetown, Salone       1
Palmetto, Georgia      1
Edmonton, CA           1
GP                     1
Name: Location, dtype: int64